In [12]:
import tensorflow as tf 
from tensorflow import keras
import warnings
warnings.filterwarnings(action="ignore")

In [3]:
(train_img,train_labels),(test_img,test_labels)=keras.datasets.cifar10.load_data()

In [4]:
train_img=train_img/255.0
test_img=test_img/255.0

In [6]:
train_img.shape

(50000, 32, 32, 3)

In [7]:
train_img=train_img.reshape(len(train_img),32,32,3)

In [8]:
test_img=test_img.reshape(len(test_img),32,32,3)

In [9]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [21]:
def build_model(hp):
    CNN_model=keras.Sequential([
        keras.layers.Conv2D(
                            filters=hp.Int("Conv_1_filter", min_value=32,max_value=128,step=16),
                            kernel_size=hp.Choice("Conv_1_Kernel_size",values=[3,5]),
                            activation="relu",
                            input_shape=(32,32,3)),
        keras.layers.Conv2D(
                            filters=hp.Int("Conv_2_filter", min_value=32,max_value=64,step=16),
                            kernel_size=hp.Choice("Conv_2_Kernel_size",values=[3,5]),
                            activation="relu"),
        keras.layers.Flatten(),
        keras.layers.Dense(
                            units=hp.Int("Units", min_value=32,max_value=128,step=16),
                            activation="relu",),
        keras.layers.Dense(10,activation="softmax")
    ])
    CNN_model.compile(optimizer=keras.optimizers.Adam(hp.Choice("Learning_Rate",values=[.001,.0001])),
                     loss="sparse_categorical_crossentropy",
                     metrics=["accuracy"])
    return CNN_model

In [22]:
tuner_search=RandomSearch(hypermodel=build_model,
                          objective="accuracy",max_trials=2,
                          directory="output",project_name="Cifar10")

In [23]:
tuner_search.search(train_img,train_labels,epochs=2)


Search: Running Trial #1

Hyperparameter    |Value             |Best Value So Far 
Conv_1_filter     |80                |?                 
Conv_1_Kernel_size|3                 |?                 
Conv_2_filter     |32                |?                 
Conv_2_Kernel_size|5                 |?                 
Units             |80                |?                 
Learning_Rate     |0.0001            |?                 

Epoch 1/2
1563/1563 [==============================] - 557s 356ms/step - loss: 1.7652 - accuracy: 0.3620
Epoch 2/2
1563/1563 [==============================] - 533s 341ms/step - loss: 1.2923 - accuracy: 0.5413

Trial 1 Complete [00h 18m 11s]
accuracy: 0.5550000071525574

Best accuracy So Far: 0.5550000071525574
Total elapsed time: 00h 18m 11s

Search: Running Trial #2

Hyperparameter    |Value             |Best Value So Far 
Conv_1_filter     |96                |80                
Conv_1_Kernel_size|3                 |3                 
Conv_2_filter     |48         

In [24]:
model=tuner_search.get_best_models(num_models=1)[0]

In [25]:
model.fit(train_img,train_labels,epochs=4,initial_epoch=2)

Epoch 3/4
1563/1563 [==============================] - 449s 287ms/step - loss: 1.1241 - accuracy: 0.6086
Epoch 4/4
1563/1563 [==============================] - 450s 288ms/step - loss: 1.0356 - accuracy: 0.6418
